In [ ]:
import os

import tensorflow as tf

from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l1_l2


from skimage.io import imread

from sklearn.model_selection import train_test_split


import numpy as np

import random

import shutil

from collections import Counter

In [ ]:
def create_labeled_list(input_img):

    folders = os.listdir(input_img)
       
    labeled_image_list = [(os.path.join(input_img, folder, image), folder) 
                          for folder in folders 
                          for image in os.listdir(os.path.join(input_img, folder))
                         ]

    img_list, label_list = zip(*labeled_image_list)
    
    return img_list, label_list

In [ ]:
def create_folder_structure(base_path):
     
    for folder in ['train', 'valid', 'test']:
        for folder1 in ['Barren', 'Cultivated', 'Developed', 'Forest', 'Herbaceous', 'Shrub']:
            pathname = os.path.join(base_path, folder, folder1)
            if not os.path.exists(pathname):
                os.makedirs(pathname)
                print("Created: {0}".format(pathname))
            else:
                print("Path {0} exists".format(pathname))


In [ ]:
def copy_images(image_list, label_list, base_folder):

    for source_path, label in zip(image_list, label_list):
        filename = source_path[source_path.rfind('/')+1: len(source_path)]
        dest_path = os.path.join(base_folder, label, filename)
        print("Copying {0} to {1}".format(source_path, dest_path))
        shutil.copyfile(source_path, dest_path)  
       
    

In [ ]:
def create_image_label_lists(input_img, ratio):
    img_list, label_list = create_labeled_list(input_img)

    img_list1, img_list2, \
    label_list1, label_list2 = train_test_split(img_list, label_list,
                                               test_size=ratio,
                                               random_state=0,
                                               stratify=label_list)

    img_list = img_list2
    label_list = label_list2

    train_imgs, valid_imgs, \
    train_labels, valid_labels = train_test_split(img_list, label_list,
                                               test_size=0.2,
                                               random_state=0,
                                               stratify=label_list)

    img_list = valid_imgs
    label_list = valid_labels

    valid_imgs, test_imgs, \
    valid_labels, test_labels = train_test_split(img_list, label_list,
                                               test_size=0.3,
                                               random_state=0,
                                               stratify=label_list)

    return train_imgs, train_labels, valid_imgs, valid_labels, test_imgs, test_labels

In [ ]:
input_img = '../../../Datasets/aerialimages/train'

train_imgs, train_labels, valid_imgs, valid_labels, test_imgs, test_labels = \
    create_image_label_lists(input_img, 0.1)

In [ ]:
print(len(train_imgs))
print(len(valid_imgs))
print(len(test_imgs))
print(len(train_labels))
print(len(valid_labels))
print(len(train_labels))

In [ ]:
print(Counter(train_labels))
print(Counter(valid_labels))
print(Counter(test_labels))

In [ ]:
base_folder = '../../../Datasets/aerialsmall'

create_folder_structure(base_folder)

In [ ]:
base_folder = '../../../Datasets/aerialsmall/train'
copy_images(train_imgs, train_labels, base_folder)

In [ ]:
base_folder = '../../../Datasets/aerialsmall/valid'
copy_images(valid_imgs, valid_labels, base_folder)

In [ ]:
base_folder = '../../../Datasets/aerialsmall/test'
copy_images(test_imgs, test_labels, base_folder)